# バイナリファイルでセーブ

### numpyでバイナリ化
- numpy.savezはヘッダーが保存できないようだ
    - ヘッダーをsavez用の辞書のkeyにする

In [2]:
import numpy as np

MAX_FILES = 3
COLUMNS = 2
ROWS = 2

# np.savezは拡張子なしでも.npzが付加されるが付けていても問題ない
DATA_FILE = 'data_noheader.npz'

# ndarray用の辞書
data_dict = {}

for i in range(MAX_FILES):
    temp_arr = np.random.rand(COLUMNS, ROWS)
    #ヘッダー兼keyの作成
    temp_key = f'header {i}\n{np.sum(temp_arr)}'
    data_dict[temp_key] = temp_arr
    
print(f'data =\n{data_dict}')
# NOTE:辞書を**で展開
np.savez(DATA_FILE, **data_dict)

# ファイルのロード
loaded_data = np.load(DATA_FILE)
print(type(loaded_data))

# テキストでセーブ
# NOTE:zipでヘッダーとdataを呼び出しても順番は保存した順番になっているようだ
for i, h in enumerate(loaded_data):
    print(f'{h}, {loaded_data[h]}')
    np.savetxt(f'data_{i}.dat', loaded_data[h], header=f'data_{i}.dat\n{h}' , delimiter='\t')


data =
{'header 0\n1.1998195168723877': array([[0.08064936, 0.90987882],
       [0.04958796, 0.15970337]]), 'header 1\n1.6208486961624127': array([[0.64222819, 0.18349195],
       [0.70879206, 0.08633648]]), 'header 2\n1.3010616033972413': array([[0.27973157, 0.55322207],
       [0.0269196 , 0.44118837]])}
<class 'numpy.lib.npyio.NpzFile'>
header 0
1.1998195168723877, [[0.08064936 0.90987882]
 [0.04958796 0.15970337]]
header 1
1.6208486961624127, [[0.64222819 0.18349195]
 [0.70879206 0.08633648]]
header 2
1.3010616033972413, [[0.27973157 0.55322207]
 [0.0269196  0.44118837]]


### 容量圧縮
- savez_compressedを使用する

In [ ]:
for i in range(MAX_FILES):
    temp_arr = np.random.rand(COLUMNS, ROWS)
    #ヘッダー兼keyの作成
    temp_key = f'header {i}\n{np.sum(temp_arr)}'
    data_dict[temp_key] = temp_arr
    
print(f'data =\n{data_dict}')
# NOTE:辞書を**で展開
np.savez_compressed(DATA_FILE, **data_dict)



### ヘッダー別ファイルバージョン
- keyにファイル名を付けたいとき等はヘッダーを別ファイルに
- dataの辞書ファイルをセーブ前に操作する場合は登録順がヘッダーファイルと変わる可能性がある
    - 安全策はヘッダーにキーワードを仕込んでre.search等で一致するものを選ぶ

In [6]:
import numpy as np

MAX_FILES = 3
COLUMNS = 2
ROWS = 2

# np.savezは拡張子なしでも.npzが付加されるが付けていても問題ない
DATA_FILE = 'data_noheader.npz'
# バイナリセーブ後にテキストで解凍するためのヘッダーファイル
HEADER_FILE = 'header.npy'

# ndarray用の辞書
data_dict = {}
headers = []

for i in range(MAX_FILES):
    temp_arr = np.random.rand(COLUMNS, ROWS)
    # ndarrayにkeyをつける
    data_dict[f'data_{i}_noheader'] = temp_arr
    
    #ヘッダーの作成
    temp_header = f'header {i}\n{np.sum(temp_arr)}'
    headers.append(temp_header)
    
print(f'feaders = \n{headers}')
print(f'data =\n{data_dict}')
# NOTE:辞書を**で展開
np.savez(DATA_FILE, **data_dict)
np.save(HEADER_FILE, headers)

# ファイルのロード
loaded_headers = np.load(HEADER_FILE)
loaded_data = np.load(DATA_FILE)
print(type(loaded_headers))
print(type(loaded_data))

# テキストでセーブ
# NOTE:zipでヘッダーとdataを呼び出しても順番は保存した順番になっているようだ

for h, d in zip(loaded_headers, loaded_data):
    print(f'{h}, {d}, {loaded_data[d]}')
    np.savetxt(f'{d}.dat', loaded_data[d], header=f'{d}.dat\n{h}' , delimiter='\t')


feaders = 
['header 0\n1.8731090373496064', 'header 1\n2.2684331176327914', 'header 2\n1.5110995566422347']
data =
{'data_0_noheader': array([[0.17785748, 0.61633583],
       [0.6571018 , 0.42181393]]), 'data_1_noheader': array([[0.70131442, 0.64028112],
       [0.83175726, 0.09508032]]), 'data_2_noheader': array([[0.46432128, 0.0743831 ],
       [0.26012692, 0.71226826]])}
<class 'numpy.ndarray'>
<class 'numpy.lib.npyio.NpzFile'>
header 0
1.8731090373496064, data_0_noheader, [[0.17785748 0.61633583]
 [0.6571018  0.42181393]]
header 1
2.2684331176327914, data_1_noheader, [[0.70131442 0.64028112]
 [0.83175726 0.09508032]]
header 2
1.5110995566422347, data_2_noheader, [[0.46432128 0.0743831 ]
 [0.26012692 0.71226826]]
